### Load Dataset

In [1]:
# LOAD CSV dataset
import pandas as pd
import numpy as np

# Load the dataset
df_filled_ml_copy = pd.read_csv('bed_cleaned_data_2025.csv') #'bed_cleaned_data.csv', isthe old 2024 data
df_filled_ml_copy.drop(columns=['Unnamed: 0'], inplace=True)
#set Item_NO to 8 digits with zfill
df_filled_ml_copy['Item_NO'] = df_filled_ml_copy['Item_NO'].apply(lambda x: str(x).zfill(8))

df_original = df_filled_ml_copy.copy()

df_filled_ml_copy.drop(columns=['Item_NO'], inplace=True)

df_filled_ml_copy.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2781 entries, 0 to 2780
Data columns (total 9 columns):
 #   Column             Non-Null Count  Dtype 
---  ------             --------------  ----- 
 0   Product_Name       2781 non-null   object
 1   length             2781 non-null   int64 
 2   width              2781 non-null   int64 
 3   Product_Type_Name  2781 non-null   object
 4   material           2781 non-null   object
 5   PA_Name            2781 non-null   object
 6   VALDES_TEXT_EN     2781 non-null   object
 7   country_code       2781 non-null   object
 8   language_code      2781 non-null   object
dtypes: int64(2), object(7)
memory usage: 195.7+ KB


### Preprocess text data

In [2]:
import re
import string
import pandas as pd
import nltk
from nltk.corpus import stopwords

# Install NLTK stopwords
nltk.download('stopwords')
stop_words = set(stopwords.words('english'))

def preprocess_text(text):
    # Convert to lowercase
    text = text.lower()
    # Remove punctuation
    text = text.translate(str.maketrans('', '', string.punctuation))
    # Remove numbers
    text = re.sub(r'\d+', '', text)
    # Remove stop words
    text = ' '.join([word for word in text.split() if word not in stop_words])
    # Remove extra whitespace
    text = text.strip()
    return text

# Assuming df_filled_ml_copy is already defined
df_no_dimensions = df_filled_ml_copy.drop(columns=['length', 'width'])
df_no_dimensions.columns = [col.lower() for col in df_no_dimensions.columns]

df_prepocess = pd.DataFrame()
for column in df_no_dimensions.columns:
    df_prepocess[column] = df_no_dimensions[column].apply(preprocess_text)

print(df_prepocess.head())

  product_name  product_type_name              material  \
0       nordli  bed frame storage  wood including board   
1       utåker      stackable bed            solid wood   
2       idanäs  bed frame storage  wood including board   
3    kleppstad          bed frame                 metal   
4    vevelstad          bed frame                 metal   

                                      pa_name     valdes_text_en country_code  \
0  bed frames beds non upholstered headboards              white           se   
1                    flexible sleep solutions               pine           se   
2  bed frames beds non upholstered headboards              white           se   
3                   bed upholstered headboard  whitevissle beige           se   
4  bed frames beds non upholstered headboards              white           se   

  language_code  
0            en  
1            en  
2            en  
3            en  
4            en  


[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\QUTRA9\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


### Similarity Clustering using Cosine similarity

In [6]:
import pandas as pd
from sentence_transformers import SentenceTransformer
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.metrics.pairwise import cosine_similarity
import numpy as np

# Load your DataFrame
df = df_prepocess.copy()
df.drop(columns=['language_code', 'country_code'], inplace=True)
df['length'] = df_filled_ml_copy['length']
df['width'] = df_filled_ml_copy['width']

# Convert the DataFrame to a list of strings
text_list = df_prepocess.apply(lambda row: ' '.join(row.values.astype(str)), axis=1).tolist()

# Step 1: Use SentenceTransformer for text data
model = SentenceTransformer('sentence-transformers/all-MiniLM-L12-v2')
text_embeddings = model.encode(text_list)

# Step 2: Use StandardScaler for length and width
scaler = StandardScaler()
length_width_scaled = scaler.fit_transform(df[['length', 'width']])

# Step 3: One-hot encode the product type
encoder = OneHotEncoder()
product_type_encoded = encoder.fit_transform(df[['product_type_name']]).toarray()

# Step 4: Combine all features
combined_features = np.hstack((text_embeddings, length_width_scaled, product_type_encoded))

# Step 5: Compute similarity scores
similarity_matrix = cosine_similarity(combined_features)

# Use absolute values of the similarity scores
similarity_matrix = np.abs(similarity_matrix)

# Step 6: Group data points based on similarity scores
threshold = 0.8  # Adjust the threshold parameter as needed
cluster_ids = [-1] * len(similarity_matrix)
current_cluster_id = 0

for i in range(len(similarity_matrix)):
    if cluster_ids[i] == -1:  # If not yet assigned to a cluster, then this will assign a new cluster
        cluster_ids[i] = current_cluster_id
        for j in range(i + 1, len(similarity_matrix)):
            if similarity_matrix[i][j] > threshold:
                cluster_ids[j] = current_cluster_id
        current_cluster_id += 1

# Add cluster_id column to the DataFrame
df['cluster_id'] = cluster_ids


In [8]:
# Describe the original clusters
print(df['cluster_id'].value_counts())

print(df['cluster_id'].value_counts().describe())

cluster_id
29     460
11     320
105    240
107    207
35     160
      ... 
85       1
62       1
83       1
76       1
84       1
Name: count, Length: 108, dtype: int64
count    108.000000
mean      25.750000
std       63.935587
min        1.000000
25%        1.750000
50%        4.500000
75%       19.250000
max      460.000000
Name: count, dtype: float64


Refine clusters by product_type_name

In [5]:
# Step 7: Refine clusters by grouping them according to their product_type_name
refined_cluster_ids = [-1] * len(df)
current_refined_cluster_id = 0

for cluster_id in set(cluster_ids):
    cluster_indices = [i for i, x in enumerate(cluster_ids) if x == cluster_id]
    product_type_names = df.iloc[cluster_indices]['product_type_name'].unique()
    
    for product_type_name in product_type_names:
        product_indices = df[(df['cluster_id'] == cluster_id) & (df['product_type_name'] == product_type_name)].index
        for idx in product_indices:
            refined_cluster_ids[idx] = current_refined_cluster_id
        current_refined_cluster_id += 1

# Add refined cluster_id column to the DataFrame
df['refined_cluster_id'] = refined_cluster_ids

# Print DataFrame with refined_cluster_id
print(df[['cluster_id', 'refined_cluster_id']])

      cluster_id  refined_cluster_id
0              0                   0
1              1                   1
2              0                   0
3              2                   2
4              2                   2
...          ...                 ...
2776         105                 142
2777         107                 144
2778         107                 144
2779         105                 142
2780         105                 142

[2781 rows x 2 columns]


Refine step 7 further by product_name (the old step here was for valid design but now changed to product_name)

In [26]:
# Step 8: Further refine clusters by grouping them according to their product_name
final_cluster_ids = [-1] * len(df)
current_final_cluster_id = 0

for refined_cluster_id in set(refined_cluster_ids):
    refined_cluster_indices = [i for i, x in enumerate(refined_cluster_ids) if x == refined_cluster_id]
    product_names = df.iloc[refined_cluster_indices]['product_name'].unique()
    
    for product_name in product_names:
        valdes_indices = df[(df['refined_cluster_id'] == refined_cluster_id) & (df['product_name'] == product_name)].index
        for idx in valdes_indices:
            final_cluster_ids[idx] = current_final_cluster_id
        current_final_cluster_id += 1

# Add final cluster_id column to the DataFrame
df['final_cluster_id'] = final_cluster_ids

insert item_no back to dataframe

In [11]:
# add Item_NO back to the dataframe
df['item_no'] = df_original['Item_NO']

Statistics description

In [28]:
# Describe the refined clusters
print(df['refined_cluster_id'].value_counts())
print(df['refined_cluster_id'].value_counts().describe())

refined_cluster_id
11     320
142    240
144    207
72     160
23     119
      ... 
49       1
54       1
66       1
125      1
99       1
Name: count, Length: 145, dtype: int64
count    145.000000
mean      19.179310
std       42.542735
min        1.000000
25%        2.000000
50%        4.000000
75%       19.000000
max      320.000000
Name: count, dtype: float64


In [27]:
# Describe the final clusters
print(df['final_cluster_id'].value_counts())
print(df['final_cluster_id'].value_counts().describe())

final_cluster_id
28     288
223     90
160     88
187     80
279     66
      ... 
106      1
234      1
231      1
218      1
246      1
Name: count, Length: 292, dtype: int64
count    292.000000
mean       9.523973
std       21.311052
min        1.000000
25%        2.000000
50%        4.000000
75%        9.000000
max      288.000000
Name: count, dtype: float64


### Forcefully Rearrange grouping to 6 members in a group
-----------------------------------

In [35]:
# Step 9: keep number of members in each refined cluster to 6
df_final_refined = pd.DataFrame()
for cluster_id in df['refined_cluster_id'].unique():
    cluster_data = df[df['refined_cluster_id'] == cluster_id]
    if len(cluster_data) > 6:
        cluster_data = cluster_data.sample(n=6)
    df_final_refined = pd.concat([df_final_refined, cluster_data])

In [13]:
# Step 10: do step 9 for each final_cluster to 6
df_final = pd.DataFrame()
for cluster_id in df['final_cluster_id'].unique():
    cluster_data = df[df['final_cluster_id'] == cluster_id]
    if len(cluster_data) > 6:
        cluster_data = cluster_data.sample(n=6)
    df_final = pd.concat([df_final, cluster_data])
    
# Print the final DataFrame
print(df_final)

     product_name         product_type_name              material  \
2          idanäs         bed frame storage  wood including board   
2052       platsa         bed frame storage  wood including board   
664       brimnes         bed frame storage  wood including board   
542       brimnes         bed frame storage  wood including board   
66         platsa         bed frame storage  wood including board   
...           ...                       ...                   ...   
2741    stavanger  wooden base spr mattress  wood including board   
2587    stavanger  wooden base spr mattress  wood including board   
2688    stavanger  wooden base spr mattress  wood including board   
2635       skårer  wooden base spr mattress  wood including board   
2740       skårer  wooden base spr mattress  wood including board   

                                         pa_name             valdes_text_en  \
2     bed frames beds non upholstered headboards                      white   
2052  bed fra

------------------

In [ ]:
# value counts for refined clusters
print(df_final_refined['refined_cluster_id'].value_counts())

# describe the refined clusters
print(df_final_refined['refined_cluster_id'].value_counts().describe())

# plot histogram of number of products in each cluster
import seaborn as sns
import matplotlib.pyplot as plt

sns.histplot(df_final_refined['refined_cluster_id'].value_counts(), kde=True)
# title and labels
plt.title('Number of Products in Groups')
plt.xlabel('Number of Products')
plt.ylabel('Number of Groups')

plt.grid(True)
plt.show()

# print count of groups that have corresponding number of products
df_final_refined['refined_cluster_id'].value_counts().value_counts()

In [ ]:
# Value counts of final clusters
print(df_final['final_cluster_id'].value_counts())

# Describe the final clusters
print(df_final['final_cluster_id'].value_counts().describe())

In [ ]:
# plot histogram of number of products in each final_cluster
import seaborn as sns
import matplotlib.pyplot as plt

sns.histplot(df_final['final_cluster_id'].value_counts(), kde=True)
# title and labels
plt.title('Number of Products in Groups')
plt.xlabel('Number of Products')
plt.ylabel('Number of Groups')

plt.grid(True)
plt.show()

# print count of groups that have corresponding number of products
df_final['final_cluster_id'].value_counts().value_counts()



inspecting group with 1 product

In [24]:
# Create a boolean Series where the count of each final_cluster_id is 1
single_member_clusters = df_final['final_cluster_id'].value_counts() == 1

# Filter df_final to get rows where final_cluster_id has only one member
single_df = df_final[df_final['final_cluster_id'].isin(single_member_clusters[single_member_clusters].index)]
